## Accessing Sentinel-2 L2A data with the Planetary Computer STAC API

The [Planetary Computer Hub](https://planetarycomputer.microsoft.com/compute) is pre-configured to use your API key.


In [176]:
### Environment setup
from pystac_client import Client
import planetary_computer as pc
import os
import fsspec

# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value
    
container = 'ana-data'

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY'],\
                 'connection_string': os.environ['CONNECTION_STRING']}

fs = fsspec.filesystem('az',\
                       account_name=storage_options['account_name'],\
                       account_key=storage_options['account_key'])   

In [177]:
import sys
sys.path.append('/content')
from src.fluvius import WaterData, WaterStation
%load_ext autoreload
%autoreload 2

import pandas as pd
import fsspec
import numpy as np
import geopandas as gpd

import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [178]:
data_source = 'ana'
container = f'{data_source}-data'
ds = WaterData(data_source, container, storage_options)
ds.get_source_df()
buffer_distance = 500 #in meters
ds.apply_buffer_to_points(buffer_distance)
ds.generate_map()
ds.plot_map

/usr/local/lib/python3.8/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [ ]:
for station in ds.df.iloc[16:]['site_no']:
    ds.get_station_data(station)
    ds.station[station].build_catalog()
    
    cloud_thr = 80
    day_tolerance = 8
    if ds.station[station].catalog is None:
        print('No matching images! Skipping...')
        continue
    else:
        ds.station[station].get_cloud_filtered_image_df(cloud_thr)
        ds.station[station].merge_image_df_with_samples(day_tolerance)
        ds.station[station].get_reflectances()
        
        outfilename = f'az://{ds.container}/stations/{station}/{station}_processed.csv'
        ds.station[station].merged_df.to_csv(outfilename,index=False,storage_options=ds.storage_options)
        #let's do the write_chips_to_blob?
        print('writing chips!')
        ds.station[station].write_tiles_to_blob(working_dirc='/tmp')


building catalog for station 28240000 with sentinel-2-l2a!
195 Items found
writing chips!
building catalog for station 28300000 with sentinel-2-l2a!
393 Items found
writing chips!
building catalog for station 28850000 with sentinel-2-l2a!
357 Items found


/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:162: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


writing chips!
building catalog for station 29050000 with sentinel-2-l2a!
320 Items found


In [22]:
import rasterio
from rasterio import features
from rasterio import warp
from rasterio import windows

scene_query = ds.station[station].merged_df.iloc[3]
visual_href = pc.sign(scene_query['visual-href'])
scl_href = pc.sign(scene_query['scl-href'])
with rasterio.open(scl_href) as ds:    
    aoi_bounds = features.bounds(ds.station[station].area_of_interest)
    warped_aoi_bounds = warp.transform_bounds('epsg:4326', ds.crs, *aoi_bounds)
    aoi_window = windows.from_bounds(transform=ds.transform, *warped_aoi_bounds)
    band_data = ds.read(window=aoi_window)
    scl = band_data[0].repeat(2, axis=0).repeat(2, axis=1)

with rasterio.open(visual_href) as ds:    
    aoi_bounds = features.bounds(area_of_interest)
    warped_aoi_bounds = warp.transform_bounds('epsg:4326', ds.crs, *aoi_bounds)
    aoi_window = windows.from_bounds(transform=ds.transform, *warped_aoi_bounds)
    band_data = ds.read(window=aoi_window)
    img = Image.fromarray(np.transpose(band_data, axes=[1, 2, 0]))
    w = img.size[0]; h = img.size[1]; aspect = w/h
    #target_w = img.size[0]*2; target_h = (int)(target_w/aspect)
    target_w = scl.shape[1]; target_h = scl.shape[0]
    img = img.resize((target_w,target_h),Image.BILINEAR)
    

f, ax = plt.subplots(1,4, figsize=(20,20))
cloud_mask = scl>7
water_mask = ((scl==6) | (scl==2))
ax[0].imshow(img)
ax[1].imshow(scl, cmap='Accent')
ax[2].imshow(cloud_mask)
ax[3].imshow(water_mask, cmap='Blues')
    #we can download the image chips here and place them in the usgs-data/{site_no}/

NameError: name 'area_of_interest' is not defined

In [109]:
np.mean(np.array(img)[water_mask],axis=0)

array([58.03409091, 57.53787879, 36.66287879])

rasterio gives us data band-interleave format; transpose to pixel interleave, and downscale the image data for plotting. Given the bounds, we will select the a nxn image around our point of interest

We might be able to create a small semantic segmentation model using our classification layer and then run that first on a scene from landsat...

In [ ]:
f, ax = plt.subplots(1,1, figsize=(20,20))
mask = water_mask[..., np.newaxis] * np.ones(np.array(img).shape).astype('bool')
#water = np.ma.MaskedArray(np.array(img), mask=~mask)
water = np.array(img)
water[~mask] =255 
ax.imshow(water)

In [4]:
#we might add a line to the gdf to include total number of cloud free chips?
#task is to open each site_no and the get the df of the estimates and then look at the cloud cover average
date_range_list = []
cloudless_count_list = []
site_no_list = []
for i, station in gdf.iterrows():
    site = str(station.site_no).zfill(8)
    url = f'az://{container}/stations/{site}/{site}.csv' #zero pad 8 digits for site_no
    try:
        df = pd.read_csv(url, storage_options=storage_options) 
        df['Date-Time'] = pd.to_datetime(df['Date-Time'])
        df = df.sort_values(by='Date-Time')
        if len(df)>0:
            #Define the aoi and time range to filter images with. 
            area_of_interest, time_of_interest = get_spacetime_bounds(station)
            cloud_thr = 5
            cloud_chip_count = len(df[df['Chip Cloud Pct']<cloud_thr])
            cloudless_count_list.append(cloud_chip_count)
            date_range_list.append(time_of_interest)
            site_no_list.append(int(site))
    except:
        print(f'{url} does not exist')
    
meta = pd.DataFrame({'site_no':site_no_list, 'cloudless_img_count':cloudless_count_list, 'date_range':date_range_list})
station_url = f'az://{container}/station_img_metadata.csv'
out = gdf.merge(meta, on='site_no').to_csv(station_url, storage_options=storage_options, index=False )

az://itv-data/stations/0000ITV1/0000ITV1.csv does not exist
az://itv-data/stations/0000ITV2/0000ITV2.csv does not exist
az://itv-data/stations/0000ITV3/0000ITV3.csv does not exist
az://itv-data/stations/0000ITV5/0000ITV5.csv does not exist
az://itv-data/stations/0000ITV6/0000ITV6.csv does not exist
az://itv-data/stations/0000ITV7/0000ITV7.csv does not exist
az://itv-data/stations/0000ITV8/0000ITV8.csv does not exist
az://itv-data/stations/0000ITV9/0000ITV9.csv does not exist
az://itv-data/stations/000ITV10/000ITV10.csv does not exist
az://itv-data/stations/000ITV11/000ITV11.csv does not exist
az://itv-data/stations/000ITV12/000ITV12.csv does not exist
az://itv-data/stations/000ITV13/000ITV13.csv does not exist
az://itv-data/stations/000ITV14/000ITV14.csv does not exist
az://itv-data/stations/000ITV15/000ITV15.csv does not exist
az://itv-data/stations/000ITV16/000ITV16.csv does not exist


ValueError: arrays must all be same length

In [5]:
np.sum(meta.cloudless_img_count)
#ana data extremely sparse for cloud free!
#trigger for planet purchase....

NameError: name 'meta' is not defined